In [1]:
# func_string = '''FUNCTION GLIQAL    300.00  -23.8463314*T*LN(T)
# -.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          3000  N!

# FUNCTION GLIQCA    300.00  -23.8463314*T*LN(T)
# -.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          1234.93  Y
# -15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 1500 Y
# 200000000 ; 3000  N !

# FUNCTION GHSERSAGG1    300.00  -23.8463314*T*LN(T)
# -.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          1234.93  Y
# -15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 3000   N !

# GHSERAGG3    300.00  -23.8463314*T*LN(T)
# -.001790585*T**2-3.98587E-07*T**3-12011*T**(-1);          1234.93  Y
# -15095.252+190.266404*T-33.472*T*LN(T)+1.412E+29*T**(-9); 3000   N !

# $ -----------------------------------------------------------------------------
# $ PARAMETERS FOR LIQUID PHASE
# $ -----------------------------------------------------------------------------
#   PHASE LIQUID:L %C 1 1.0 !
#   CONSTITUENT LIQUID :AL,AL2CA,CA: ! 
#   PARAMETER L(LIQUID,AL;0) 298.15 +GLIQAL;                            6000 N !
#   PARAMETER L(LIQUID,CA;0) 298.15 +GLIQCA;                            6000 N !
#   PARAMETER G(LIQUID,AL2CA;0) 298.15
#   -77214+46.424*T+2*GLIQAL+GLIQCA;                                   6000 N !
#   PARAMETER L(LIQUID,AL,CA;0) 298.15 -68842+25.372*T;                 6000 N !
#   PARAMETER L(LIQUID,AL2CA,AL;0) 298.15 -30351+2.939*T;               6000 N !
#   PARAMETER L(LIQUID,AL2CA,CA;0) 298.15 -52132+0.848*T;               6000 N !
# $ -----------------------------------------------------------------------------
# $ PARAMETERS FOR Another PHASE
# $ -----------------------------------------------------------------------------
#   PHASE ANOTHER:L % 2 1.0 1.1  !
#   CONSTITUENT ANOTHER :AL,AL2CA,CA:AL,CA: ! 
#   PARAMETER L(ANOTHER,AL;0) 298.15 +GLIQAL*10;                            6000 N !
#   PARAMETER L(ANOTHER,CA;0) 298.15 +GLIQCA;                            6000 N !
#   PARAMETER G(ANOTHER,AL2CA;0) 298.15
#    -77214+46.424*T+2*GLIQAL+GLIQCA;                                   6000 N !
#   PARAMETER L(ANOTHER,AL,CA;0) 298.15 -68842+25.372*T;                 6000 N !
#   PARAMETER L(ANOTHER,AL2CA,AL;0) 298.15 -30351+2.939*T;               6000 N !
#   PARAMETER L(ANOTHER,AL2CA,CA;0) 298.15 -52132+0.848*T;               6000 N !
# '''

with open('databases/NIMS_alsi.TDB') as file:
    db_string = file.read()

In [17]:
from src.functions import thermo_func
from src.parameter import Parameter
from src.helper_functions import *
from src.phase import Phase
from src.models import Ideal_solution_model

import re

class System:
    def __init__(self, string):
        self.database_string = remove_comments(string)
        self.database_lines = split_lines(self.database_string)

        self.function_list, self.param_list, self.phase_list = self._parse_database(self.database_lines)

    def _parse_database(self, lines):
        
        temp_function_list = []
        temp_param_list = []
        temp_phase_list = []

        for line in lines:
            if 'FUNCTION' in line:
                temp_function_list.append(thermo_func(line))
        for line in lines:
            if 'PARAMETER' in line:
                # print('Parameter: ', line)
                temp_param_list.append(Parameter(line, temp_function_list))
        for line in lines:
            if 'PHASE' in line:
                match = re.search(r'\s?+PHASE\s(\S+):?([G, A, Y, L, I, F, B])?\s+(\S+)\s+(\d)\s+(.+)\s+', line)
                sublattice_match = re.findall(r'\d+\.\d+', line)
                if match:
                    if sublattice_match:
                        phase_name = match.group(1)
                        g_phase_type_code = match.group(2)
                        data_type_code = match.group(3)
                        num_sub_lattices = match.group(4)
                        sub_lattice_sites_list = sublattice_match

                        temp_phase_list.append(
                            Phase(
                                phase_name,
                                g_phase_type_code,
                                data_type_code,
                                num_sub_lattices,
                                sub_lattice_sites_list,
                                temp_param_list
                            )
                        )

                        # print('test', temp_function_list)

            if 'CONSTITUENT' in line:
                print('Constitituent: ', line)
                match = re.search(r'\s+?CONSTITUENT\s{}\s:(\S+):'.format(temp_phase_list[-1].name), line)
                
                species_list = []
                if match:
                        species = match.group(1)
                        species = species.split(':')
                        sub_lattice = []

                        for i in species:
                            sub_lattice = i.split(',')
                            # print(sub_lattice)
                            species_list.append(sub_lattice)
                
                temp_phase_list[-1].set_species(species_list)
        

        return temp_function_list, temp_param_list, temp_phase_list


    def set_models(self, model_list):
        for i, phase in enumerate(self.phase_list):
            phase.set_model(model_list[i], self.T)


    def set_conditions(self, temperature, pressure):

        self.T = temperature
        self.P = pressure
        for phase in self.phase_list:
            phase.calc_parameters(temperature)

    def calc_gibbs(self, const_list):
        self.gibbs = 0

        for i, phase in enumerate(self.phase_list):
            self.gibbs = self.gibbs + (phase.calc_model(const_list[i]))

        return self.gibbs




In [16]:
system = System(db_string)

system.set_conditions(2900, 101325)
model = Ideal_solution_model()
models = [model, model, model]

system.set_models(models, 1000000)

const_array = [
    [0.1, 0.5],
    [5, 10], 
    [0, 10]
]

system.calc_gibbs(const_array)

Constitituent:  CONSTITUENT LIQUID:L :AL,SI:  
Constitituent:  CONSTITUENT DIAMOND_A4  :AL,SI:  
Constitituent:  CONSTITUENT FCC_A1  :AL,SI:  
[0.1, 0.5] [<src.parameter.Parameter object at 0x00000178EE4C9670>, <src.parameter.Parameter object at 0x00000178EE4CBDD0>, <src.parameter.Parameter object at 0x00000178EE4CB860>, <src.parameter.Parameter object at 0x00000178EE4C9490>, <src.parameter.Parameter object at 0x00000178EE4CBA70>]
[5, 10] [<src.parameter.Parameter object at 0x00000178EE4CADE0>, <src.parameter.Parameter object at 0x00000178EE4C8BC0>, <src.parameter.Parameter object at 0x00000178EE4C9160>]
[0, 10] [<src.parameter.Parameter object at 0x00000178EE4CA2D0>, <src.parameter.Parameter object at 0x00000178EE4CBA10>, <src.parameter.Parameter object at 0x00000178EE4C9340>]


np.float64(-3855161.935218707)

In [4]:
line ='  Constitituent:  CONSTITUENT FCC_A1  :AL,SI:'  

match = re.search(r'\s+?CONSTITUENT\sFCC_A1\s+:(\S+):', line)
match.groups()

('AL,SI',)